In [1]:
%load_ext autoreload
%autoreload 2

First run `chargpt.py` then this notebook

In [2]:
from mingpt.utils import set_seed
import torch
from mingpt.model import GPT
import projects.chargpt.chargpt as chargpt

set_seed(3407)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
config = chargpt.get_config()
text = open("input.txt", "r").read()
train_dataset = chargpt.CharDataset(config.data, text)
config.model.vocab_size = train_dataset.get_vocab_size()
config.model.block_size = train_dataset.get_block_size()
model = GPT(config.model).to(device)
model.eval()
weights = torch.load("out/chargpt/model.pt")
model.load_state_dict(weights)

data has 77358 characters, 61 unique.
number of parameters: 2.71M


<All keys matched successfully>

In [4]:
text[:15]

'First Citizen:\n'

In [5]:
context = "Who is bob sagget?"
encode = [train_dataset.stoi[s] for s in context]
x = torch.tensor(encode, dtype=torch.long)[None, ...].to(device)

In [6]:
# We will make B generations for the same prompt
B = 5
x = x.repeat(B, 1)

In [7]:
def decode(tokens: torch.Tensor) -> str:
    return ''.join([train_dataset.itos[int(i)] for i in tokens])

In [8]:
y, log_probs = model.generate(x, 500, temperature=1.0, do_sample=True, top_k=None, do_log_prob=True)

In [9]:
print(("\n" + "-" * 64 + "\n").join([decode(i) for i in y]))

Who is bob sagget?

MENENIUS:
The custom of request you have discharged:
The people do admit you, and are summon'd
To meet anon, upon your approbation.

CORIOLANUS:
Where? at the senate-house?

SICINIUS:
There, Coriolanus.

CORIOLANUS:
May I change these garments?

SICINIUS:
You may, sir.

CORIOLANUS:
That I'll straight do; and, knowing myself again,
Repair to the senate-house.

MENENIUS:
I'll keep you company. Will you along?

BRUTUS:
We stay here for the people.

SICINIUS:
Fare you well.
He has it now, and by h
----------------------------------------------------------------
Who is bob sagget?

MENENIUS:
The common modest that express his disposition,
And follow Marcius.
O, me alone! make you a sword of me?
If these shows be not outward, which of you
But is four Volsces? none of you but is
Able to bear against the great Aufidius
A shield as hard as his. A certain number,
Though thanks to all, must I select
from all: the rest
Shall bear the business in some other fight,
As cause will 

In [10]:
perplexity = torch.exp(log_probs.mean(-1))
perplexity

tensor([0.9972, 0.9828, 0.9945, 0.9770, 0.9972], device='cuda:0')